## Import Necessary Libraries
pandas and numpy handle dataframe and data operations.

Faker generates realistic names, dates, locations, etc.

random handles random choices.

We instantiate Faker to use throughout.

In [9]:
# Import key libraries for data generation and DataFrame handling
import pandas as pd
import numpy as np
from faker import Faker
import random
import datetime

# Initialize Faker for realistic synthetic data
fake = Faker()


## Generate Synthetic Flights Table

Generates 50 synthetically unique flights.

Flight numbers, origins, destinations, and times are randomized.

departure_date ensures plausible future flight dates.

Displays first few records for verification.

In [10]:
# Create synthetic flights data
flight_records = []
flight_numbers = [f"BA{num:03d}" for num in range(100, 200)]  # Eg: BA100 to BA199
origins = ["London", "Paris", "Berlin", "Amsterdam", "Rome", "Madrid", "Dublin"]
destinations = ["New York", "Dubai", "Singapore", "Toronto", "Sydney", "Tokyo"]

# Convert string dates to datetime.date objects
start_date = datetime.date(2025, 11, 18)
end_date = datetime.date(2025, 12, 31)

for i in range(50):  # 50 flights
    record = {
        'flight_id': i+1,
        'flight_number': random.choice(flight_numbers),
        'origin': random.choice(origins),
        'destination': random.choice(destinations),
        'departure_date': fake.date_between(start_date=start_date, end_date=end_date),
        'departure_time': fake.time(pattern='%H:%M'),
        'total_seats': random.randint(100, 300)
    }
    flight_records.append(record)

flights_df = pd.DataFrame(flight_records)
flights_df.head()


,flight_id,flight_number,origin,destination,departure_date,departure_time,total_seats
0,1,BA136,Madrid,Toronto,2025-12-18,12:09,161
1,2,BA136,Berlin,New York,2025-11-26,05:30,227
2,3,BA178,London,Sydney,2025-12-26,21:27,161
3,4,BA138,Madrid,Singapore,2025-12-12,08:46,159
4,5,BA193,Madrid,Singapore,2025-12-21,00:27,180


# Generate Synthetic Passengers Table
1100 passengers gives diversity and noise for 1000+ bookings.

Gender, age, names, passport numbers synthetic/randomized.

Deliberate missing values: randomly set 20 passport numbers to None.


Display sample for verification.

In [11]:
# Generate synthetic passenger data
passenger_records = []
for i in range(1100):  # 1100 passengers for variety/missing/duplicates
    record = {
        'passenger_id': i+1,
        'first_name': fake.first_name(),
        'last_name': fake.last_name(),
        'gender': random.choice(['Male', 'Female', 'Other']),
        'age': random.randint(18, 85),
        'passport_number': fake.bothify(text='??######')
    }
    passenger_records.append(record)

# Optionally insert deliberate missing values and duplicates
for idx in random.sample(range(1100), 20):
    passenger_records[idx]['passport_number'] = None  # Deliberate missing passports


passengers_df = pd.DataFrame(passenger_records)
passengers_df.head()


,passenger_id,first_name,last_name,gender,age,passport_number
0,1,Brian,Hernandez,Other,43,qY335122
1,2,James,Tran,Female,48,Nc321736
2,3,Joel,Adams,Male,57,PS162379
3,4,Kenneth,Clarke,Other,66,dW479681
4,5,Amy,Jordan,Other,35,Et612383


# Generate Bookings Table (Composite Keys)

Creates booking records with random passenger/flight pairs.

Ensures composite uniqueness (flight_id, passenger_id) reflecting real PK.

Adds variety, including more "Confirmed" statuses.

3% missing seat_classes for realism.

Slices to 1000 unique rows as required.

Shows sample output.

In [12]:
seat_classes = ['Economy', 'Premium', 'Business']
booking_statuses = ['Confirmed', 'Pending', 'Cancelled']
booking_records = []
booking_set = set()

start_date = datetime.date(2025, 10, 1)
end_date = datetime.date(2025, 11, 17)

for _ in range(1200):
    flight_id = random.randint(1, 50)
    passenger_id = random.randint(1, 1102)
    composite_key = (flight_id, passenger_id)
    if composite_key in booking_set:
        continue
    booking_set.add(composite_key)
    booking_date = fake.date_between(start_date=start_date, end_date=end_date)
    price_paid = round(random.uniform(80, 3000), 2)
    booking_status = random.choices(booking_statuses, weights=[0.7, 0.25, 0.05])[0]
    # Introduce 3% missing values for seat_class
    if random.random() < 0.03:
        seat_class = None
    else:
        seat_class = random.choice(seat_classes)
    booking_records.append({
        'flight_id': flight_id,
        'passenger_id': passenger_id,
        'seat_class': seat_class,
        'booking_date': booking_date,
        'price_paid': price_paid,
        'booking_status': booking_status
    })

bookings_df = pd.DataFrame(booking_records[:1000])
bookings_df.head()


,flight_id,passenger_id,seat_class,booking_date,price_paid,booking_status
0,37,591,Economy,2025-11-06,1533.52,Confirmed
1,42,1056,None,2025-11-10,2271.48,Confirmed
2,31,1060,Economy,2025-11-14,2189.76,Pending
3,4,965,Business,2025-10-07,2242.97,Confirmed
4,46,972,Premium,2025-11-16,421.30,Pending


# Generate Payments Table (Missing )
Links payments to bookings via their composite key.

3% missing payments for realism.

Matches booking price for amount.

Sample display for verification.


In [13]:
import datetime

# Set correct date objects for payment date range
payment_start_date = datetime.date(2025, 10, 1)
payment_end_date = datetime.date(2025, 11, 19)

payment_methods = ['Card', 'Cash', 'Paypal', 'Bank Transfer']
payment_statuses = ['Completed', 'Failed', 'Refunded']

payment_records = []
for i, booking in bookings_df.iterrows():
    # Not all bookings will have a payment (deliberately missing values)
    if random.random() < 0.97:  # 3% missing payments
        # Pick payment_date: either on booking_date, or random in range
        payment_date = booking['booking_date'] if random.random() < 0.85 else fake.date_between(start_date=payment_start_date, end_date=payment_end_date)
        payment_status = random.choices(payment_statuses, weights=[0.9, 0.07, 0.03])[0]
        payment_records.append({
            'payment_id': i+1,
            'flight_id': booking['flight_id'],
            'passenger_id': booking['passenger_id'],
            'payment_method': random.choice(payment_methods),
            'payment_date': payment_date,
            'amount': booking['price_paid'],
            'payment_status': payment_status
        })


payments_df = pd.DataFrame(payment_records)
payments_df.head()



,payment_id,flight_id,passenger_id,payment_method,payment_date,amount,payment_status
0,1,37,591,Cash,2025-10-03,1533.52,Completed
1,2,42,1056,Bank Transfer,2025-11-10,2271.48,Completed
2,3,31,1060,Cash,2025-11-14,2189.76,Completed
3,4,4,965,Paypal,2025-10-07,2242.97,Failed
4,5,46,972,Cash,2025-11-16,421.30,Completed


# Export DataFrames to CSV (for SQLite import)
Saves all generated tables as CSV for quick SQLite upload.

eliminates any records that could create foreign key errors in your database.

Guarantees database integrity: every booking references an existing flight and passenger; every payment references an existing booking.

Supports a seamless, error-free import process in SQLite and upholds proper relational database modeling.

In [14]:
# Clean Bookings - keep only those with real flight and passenger
bookings_df_clean = bookings_df[
    bookings_df['flight_id'].isin(flights_df['flight_id']) &
    bookings_df['passenger_id'].isin(passengers_df['passenger_id'])
]

print("Before cleaning Bookings:", len(bookings_df))
print("After cleaning Bookings:", len(bookings_df_clean))

# Create set of valid composite booking keys
valid_booking_keys = set(zip(bookings_df_clean['flight_id'], bookings_df_clean['passenger_id']))

# Clean Payments - keep only those for real bookings
payments_df_clean = payments_df[
    payments_df.apply(lambda row: (row['flight_id'], row['passenger_id']) in valid_booking_keys, axis=1)
]

print("Before cleaning Payments:", len(payments_df))
print("After cleaning Payments:", len(payments_df_clean))

# Export only the cleaned DataFrames to CSV
flights_df.to_csv('Flights.csv', index=False)
passengers_df.to_csv('Passengers.csv', index=False)
bookings_df_clean.to_csv('Bookings.csv', index=False)
payments_df_clean.to_csv('Payments.csv', index=False)


Before cleaning Bookings: 1000
After cleaning Bookings: 1000
Before cleaning Payments: 966
After cleaning Payments: 966


# Double check the missing values 

In [15]:
# Missing values
for name, df in zip(['Flights', 'Passengers', 'Bookings', 'Payments'],
                    [flights_df, passengers_df, bookings_df, payments_df]):
    print(f"\nMissing values in {name}:")
    print(df.isnull().sum())
    



Missing values in Flights:
flight_id         0
flight_number     0
origin            0
destination       0
departure_date    0
departure_time    0
total_seats       0
dtype: int64

Missing values in Passengers:
passenger_id        0
first_name          0
last_name           0
gender              0
age                 0
passport_number    20
dtype: int64

Missing values in Bookings:
flight_id          0
passenger_id       0
seat_class        34
booking_date       0
price_paid         0
booking_status     0
dtype: int64

Missing values in Payments:
payment_id        0
flight_id         0
passenger_id      0
payment_method    0
payment_date      0
amount            0
payment_status    0
dtype: int64
